In [3]:
from devtools import debug

%load_ext autoreload
%autoreload 2

!export PYTHONPATH=":./python"

In [1]:
from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

from python.ai_chains.B_2_self_query import get_query_constructor, get_retriever

In [4]:
r = get_query_constructor(
    {
        "query": "What are some sci-fi movies from the 90's directed by Luc Besson about taxi drivers"
    }
)
debug(r)

2024-06-28 00:14:43.678 | INFO     | python.ai_core.llm:get_llm:395 - get LLM : gpt_35_edenai - configurable: True
2024-06-28 00:14:44.271 | INFO     | python.ai_core.llm:get_configurable:351 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


/tmp/ipykernel_162455/997951588.py:6 <module>
    r: RunnableSequence(
        first=FewShotPromptTemplate(
            input_variables=['query'],
            examples=[
                {
                    'i': 1,
                    'data_source': (
                        '```json\n'
                        '{{\n'
                        '    "content": "Lyrics of a song",\n'
                        '    "attributes": {{\n'
                        '        "artist": {{\n'
                        '            "type": "string",\n'
                        '            "description": "Name of the song artist"\n'
                        '        }},\n'
                        '        "length": {{\n'
                        '            "type": "integer",\n'
                        '            "description": "Length of the song in seconds"\n'
                        '        }},\n'
                        '        "genre": {{\n'
                        '            "type": "string",\n'

FewShotPromptTemplate(input_variables=['query'], examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{{\n    "query": "teenager love",\n    "filter": "and(or(eq(\\"artist\\", \\"Taylor Swift\\"), eq(\\"artist\\", \\"Katy Perry\\")), lt(\\"length\\", 180), eq(\\"genre\\", \\"pop\\"))"\n}}\n```'}, {'i': 2, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attri

In [18]:
df = pd.read_excel("abbreviations.xlsx", index_col=0)
unknown = df[df.iloc[:, 0].isnull()]
unknown_list = "\n".join(unknown.index.tolist()[0:10])
first = unknown.index.to_list()[0]


system = """ 
Vous êtes expert du domaine de la formation et de l'enseignement supérieur.
Repondez uniquement en Francais.
Pour chacune des abréviations suivantes, completez par la signification si vous la connaissez.
Si vous ne savez pas, afficher "?".
N'ajoutez pas de commentaires et autres information.
"""

user = """
###  ABBREVIATIONS ###
{abrev}
### REPONSE: ###
- {first_one}: ...
- ...
"""

llm_mistral = get_llm("mistral_large_edenai")

prompt = def_prompt(system, user)
chain = prompt | llm_mistral | StrOutputParser()

# answer_mistral = chain.invoke({})

2024-06-20 19:17:24.197 | INFO     | python.ai_core.llm:get_llm:395 - get LLM : mistral_large_edenai - configurable: True


2024-06-20 19:17:24.587 | INFO     | python.ai_core.llm:get_configurable:351 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


In [36]:
llm_gpt4 = get_llm("gpt_4o_edenai")
chain = prompt | llm_gpt4 | StrOutputParser()

2024-06-20 21:16:11.348 | INFO     | python.ai_core.llm:get_llm:395 - get LLM : gpt_4o_edenai - configurable: True
2024-06-20 21:16:11.798 | INFO     | python.ai_core.llm:get_configurable:351 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


In [37]:
rep = chain.invoke({"abrev": unknown_list, "first_one": first})
print(rep)

- CAPPEI: Certificat d'Aptitude Professionnelle aux Pratiques de l'Éducation Inclusive
- MPPM: ?
- BIOGET: ?
- MTS: ?
- SIIA: ?
- EIFFALE: ?
- TROPIMUNDO: ?
- TALN: Traitement Automatique du Langage Naturel
- ESDD: ?
- ORMS: ?


In [24]:
line = "- ESDD: Education au Développement Durable"

a = line.strip("- ")
a

'ESDD: Education au Développement Durable'

In [35]:
lines = rep.split("\n")

d = dict()
for line in lines:
    print(line)
    k, _, v = line.partition(":")
    d |= {k.strip("- "): v.strip()}


df = pd.DataFrame.from_dict(d, orient="index")
df.to_excel("abbreviations2.xlsx", sheet_name="llm")

- CAPPEI: Certificat d'Aptitude Professionnelle aux Pratiques de l'Education Inclusive
- MPPM: ?
- BIOGET: ?
- MTS: Mastère Spécialisé
- SIIA: Systèmes d'Information et d'Ingénierie Avancée
- EIFFALE: ?
- TROPIMUNDO: ?
- TALN: Traitement Automatique des Langues Naturelles
- ESDD: Education au Développement Durable
- ORMS: ?


In [31]:
d

{'abbrev': 'ORMS', 'desc': '?'}

In [45]:
l = [1, 1]
for x in l or []:
    print(x)

1
1


In [46]:
l += [2, 2]
l

[1, 1, 2, 2]